<a href="https://colab.research.google.com/github/snbigft/Create_Lotto_Database_from_csv/blob/main/Inserta_datos_Lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import psycopg2
from psycopg2 import sql , extras
import json
from google.colab import drive

# Configuración de la base de datos
DB_HOST = "ep-snowy-dawn-a8omlpke-pooler.eastus2.azure.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_BcpnId62FYhS"

# Configuración de inserción en bloques
chunk = 10000  # Número de filas en la BBDD antes de cada commit
filas_por_linea_csv = 5  # Cada línea del CSV inserta 5 filas en la BBDD
lineas_por_chunk = chunk // filas_por_linea_csv  # Número de líneas del CSV por cada commit

def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

# Montar Google Drive y leer el fichero CSV
drive.mount('/content/drive')
csv_file_path = '/content/drive/My Drive/Lotto_it.csv'

# Leer el CSV con manejo de errores
try:
    df = pd.read_csv(csv_file_path, sep=';', encoding='ISO-8859-1')
    print(f"Archivo CSV cargado con éxito. Número de filas: {len(df)}")
except Exception as e:
    print(f"Error al leer el archivo CSV: {e}")

# Conectar a la base de datos
try:
    conn = get_db_connection()
    cursor = conn.cursor()
    print("Conexión a la base de datos establecida correctamente.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

# Almacenar las consultas antes de ejecutarlas en bloque
insert_values = []

# Recorrer cada fila del CSV
for index, row in df.iterrows():
    try:
        data = row['Data estr.']
        ordine = int(row['ordine'])
        ruota = row['Ruota']
        numeros = [row['1°'], row['2°'], row['3°'], row['4°'], row['5°']]

        # Preparar cinco filas para insertar en la tabla LOTTO_IT
        for posicion, numero in enumerate(numeros, start=1):
            json_data = json.dumps({
                "data": data,
                "ordine": str(ordine),
                "ruota": ruota,
                "posicion": str(posicion),
                "numero": str(numero)
            })

            # Almacenar los valores en la lista para la inserción en bloque
            insert_values.append((data, ordine, ruota, int(numero), json_data, posicion))

        # Realizar la inserción en bloque cada 'chunk' filas
        if len(insert_values) >= chunk:
            insert_query = sql.SQL("""
                INSERT INTO "lotto_it" (giorno, ordine, ruota, numero, json_data, posicion)
                VALUES %s
            """)

            psycopg2.extras.execute_values(
                cursor, insert_query.as_string(conn), insert_values, template=None, page_size=100
            )

            conn.commit()
            print(f"Commit realizado después de insertar {len(insert_values)} filas.")
            insert_values = []

    except Exception as e:
        print(f"Error procesando la fila {index+1}: {e}")

# Insertar cualquier fila restante que no alcanzó el chunk completo
if insert_values:
    insert_query = sql.SQL("""
        INSERT INTO "lotto_it" (giorno, ordine, ruota, numero, json_data, posicion)
        VALUES %s
    """)

    psycopg2.extras.execute_values(
        cursor, insert_query.as_string(conn), insert_values, template=None, page_size=100
    )

    conn.commit()
    print(f"Commit final realizado con {len(insert_values)} filas restantes.")

# Cerrar conexión
cursor.close()
conn.close()
print("Datos insertados correctamente en la tabla LOTTO_IT.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivo CSV cargado con éxito. Número de filas: 101373
Conexión a la base de datos establecida correctamente.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit realizado después de insertar 10000 filas.
Commit real